In [1]:
import io, time, json
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def read_api_key(filepath):
    """
    Read the Yelp API Key from file.
    
    Args:
        filepath (string): File containing API Key
    Returns:
        api_key (string): The API Key
    """
    
    # feel free to modify this function if you are storing the API Key differently
    with open('api_key.txt', 'r') as f:
        return f.read().replace('\n','')

In [3]:
def all_restaurants(api_key, query):
    """
    Retrieve ALL the restaurants on Yelp for a given query.

    Args:
        query (string): Search term

    Returns:
        results (list): list of dicts representing each business
    """
    
    # Write solution here
    offsetno=0
    total =0
    output=[]
    headers = {        
        "authorization": 'Bearer %s' % api_key, # for the yelp API     
    }
    while (offsetno<=total):
        params = { #parameters are case sensitive!
            "location": query,
            "limit": 20,
            "offset": offsetno,
            "categories": "restaurants"
            
        }
    
        response = requests.get('https://api.yelp.com/v3/businesses/search',
                            headers=headers, params=params)
        result = response.json()
        try:
            output.append( result['businesses'])
        except KeyError:
            print(result)
            return output
        offsetno =offsetno + 20
        total = result['total']
        print (offsetno)
        print (len(output))
    return(output)    

In [4]:
api_key = read_api_key('api_key.txt')
data = all_restaurants(api_key, 'Miami')

20
1
40
2
60
3
80
4
100
5
120
6
140
7
160
8
180
9
200
10
220
11
240
12
260
13
280
14
300
15
320
16
340
17
360
18
380
19
400
20
420
21
440
22
460
23
480
24
500
25
520
26
540
27
560
28
580
29
600
30
620
31
640
32
660
33
680
34
700
35
720
36
740
37
760
38
780
39
800
40
820
41
840
42
860
43
880
44
900
45
920
46
940
47
960
48
980
49
1000
50
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}


In [69]:
new_data = {}
new_data["name"]=[]
new_data["rating"]=[]
new_data["price"]=[]
new_data["review_count"]=[]
new_data["url"]=[]
new_data["category"]=[]
new_data["zip"]=[]
alldata=[]
for i in data:
    for each in i:
        new_data["name"].append(each["name"])
        new_data["rating"].append(each["rating"])
        new_data["url"].append(each["url"])
        try:
            new_data["price"].append(each["price"])
        except KeyError:
            new_data["price"].append(None)

        new_data["review_count"].append(each["review_count"])
        new_data["category"].append(each["categories"][0]["title"])
        new_data["zip"].append(each["location"]["zip_code"])
        alldata.append(each)
    


In [21]:
df = pd.DataFrame(new_data)
df.to_csv("Miami_Yelp.csv",index=False)

In [72]:
df_all = pd.DataFrame(alldata)
df_all.to_csv("Miami_raw.csv",index=False)

In [10]:
data[0][0]["location"]["zip_code"]

'33135'

[{'id': 'hZm7TunlrksQbgS0ssXbUg', 'alias': 'versailles-restaurant-miami', 'name': 'Versailles Restaurant', 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/XSwOCEiZjeNcDSUmgYbTFA/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/versailles-restaurant-miami?adjust_creative=e1AJwhqcU3hPEpOOJGLaFQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=e1AJwhqcU3hPEpOOJGLaFQ', 'review_count': 3324, 'categories': [{'alias': 'cuban', 'title': 'Cuban'}, {'alias': 'coffee', 'title': 'Coffee & Tea'}, {'alias': 'latin', 'title': 'Latin American'}], 'rating': 4.0, 'coordinates': {'latitude': 25.7650962116388, 'longitude': -80.2527339189549}, 'transactions': ['restaurant_reservation'], 'price': '$$', 'location': {'address1': '3555 SW 8th St', 'address2': '', 'address3': '', 'city': 'Miami', 'zip_code': '33135', 'country': 'US', 'state': 'FL', 'display_address': ['3555 SW 8th St', 'Miami, FL 33135']}, 'phone': '+13054440240', 'display_phone': '(305) 444-0240', 'distan